In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
import pandas as pd
from collections import defaultdict as _dd
import hashlib, math, bisect
import random as _rnd

# Constants 

In [2]:
SLOTS_PER_EPOCH   = 24000
EEP_FRACTION      = 1/3
EEP_SLOT          = int(SLOTS_PER_EPOCH * EEP_FRACTION)

PENALTY_FACTOR        = 2.0
RECOVERY_FACTOR       = 1.0
TOLERATED_BAD_RATIO   = 0.02
REWARD_PER_BLOCK      = 0.0001
AUTO_COMPOUND         = True
RNG_SEED              = 42
MIN_BET_AMOUNT        = 50.0
TOKEN_SCALE_INT       = 10**18
W_SKIPPED             = 1.0
W_INVALID             = 2.0
W_DELAYED             = 1.5

In [3]:
FEE_SCHEDULE_TKR: Dict[str, float] = {
    "PAY":                0.0001,
    "STAKE":              0.0010,
    "STAKE_UNDO":         0.0010,
    "REGISTER_MAIN":      1.0,
    "REGISTER_OVERFLOW":  0.1,
    "DEREGISTER_MAIN":    0.5,
    "DEREGISTER_OVERFLOW":0.05,
    "ASSIGN_OVERFLOW":    0.05,
    "UPLOAD_CONTRACT":    10.0,
    "CALL_FUNCTION":      0.01,
}

# Classes

In [4]:
@dataclass(slots=True)
class Wallet:
    address: str
    green: float = 0.0   # TKG
    red: float   = 0.0   # TKR

    def get_balance(self) -> Dict[str, float]:
        return {"green": self.green, "red": self.red}

    def deposit(self, *, green: float = 0.0, red: float = 0.0) -> None:
        if green > 0: self.green += green
        if red   > 0: self.red   += red

    def transfer(self, other: "Wallet", *, green_amount: float = 0.0, red_amount: float = 0.0) -> bool:
        if green_amount < 0 or red_amount < 0: return False
        if green_amount > self.green or red_amount > self.red: return False
        if green_amount:
            self.green -= green_amount; other.green += green_amount
        if red_amount:
            self.red   -= red_amount;   other.red   += red_amount
        return (green_amount > 0) or (red_amount > 0)

    def bet_op(self, to_main_id: str, amount: float, when_slot: int) -> dict:
        return {"kind": "STAKE", "from": self.address, "to": to_main_id,
                "amount": float(amount), "slot": when_slot}

In [5]:
@dataclass
class MainAddress(Wallet):
    delegates: Dict[str, float] = field(default_factory=dict)
    active_stake: float = 0.0
    overflows: Dict[str, "OverflowNode"] = field(default_factory=dict)

    def register_overflow(self, overflow: "OverflowNode") -> None:
        overflow.main_id = self.address
        self.overflows[overflow.address] = overflow

    def get_overflow_list(self) -> list:
        return list(self.overflows.values())

    def bet(self, from_address: str, amount: float) -> bool:
        if amount < MIN_BET_AMOUNT: return False
        self.delegates[from_address] = self.delegates.get(from_address, 0.0) + amount
        return True

    def self_bet(self, amount: float) -> bool:
        if amount < MIN_BET_AMOUNT or amount > self.green: return False
        self.green -= amount
        return self.bet("_self_", amount)

    def self_unbet(self, amount: Optional[float] = None) -> float:
        cur = self.delegates.get("_self_", 0.0)
        if cur <= 0: return 0.0
        take = cur if amount is None else max(0.0, min(amount, cur))
        if take <= 0: return 0.0
        residual = cur - take
        if residual > 0: self.delegates["_self_"] = residual
        else:            self.delegates.pop("_self_", None)
        self.green += take
        return take

    def total_delegated_now(self) -> float:
        return sum(self.delegates.values())

    def snapshot_at_eep(self) -> None:
        self.active_stake = self.total_delegated_now()

    def credit_reward(self, amount_green: float) -> None:
        """Coinbase in TKG pro-rata sulle deleghe (compounding)."""
        if amount_green <= 0: return
        total = self.total_delegated_now()
        if total <= 0:
            self.green += amount_green
            return
        for addr, stake_amt in list(self.delegates.items()):
            share = amount_green * (stake_amt / total)
            self.delegates[addr] = stake_amt + share

In [6]:
@dataclass(slots=True)
class OverflowNode(Wallet):
    main_id: str = ""
    # comportamento rete
    p_skip: float = 0.00
    p_invalid: float = 0.00
    p_delayed: float = 0.00
    # comportamento tx
    p_bad_tx: float = 0.0
    # stato
    penalty_score: float = 0.0
    effective_stake: float = 0.0
    assigned_slots: int = 0
    penalty_slots_residual: int = 0

    def behavior_outcome(self, rng) -> str:
        x = rng.random()
        if x < self.p_skip:    return "skipped"
        x -= self.p_skip
        if x < self.p_invalid: return "invalid"
        x -= self.p_invalid
        if x < self.p_delayed: return "delayed"
        return "ok"

    def slash_green(self, amount: float) -> float:
        if amount <= 0: return 0.0
        taken = min(self.green, amount)
        self.green -= taken
        return taken

In [7]:
@dataclass(slots=True)
class Transaction:
    txid: str
    kind: str
    sender: str
    recipient: str
    amount_green: float = 0.0
    amount_red: float = 0.0
    fee_paid_tkr: float = 0.0
    valid: bool = True

In [8]:
class TakamakaSim:
    def __init__(self,
                 *,
                 slots_per_epoch: int = SLOTS_PER_EPOCH,
                 eep_fraction: float = EEP_FRACTION,
                 penalty_factor: float = PENALTY_FACTOR,
                 recovery_factor: float = RECOVERY_FACTOR,
                 tolerated_bad_ratio: float = TOLERATED_BAD_RATIO,
                 reward_per_block: float = REWARD_PER_BLOCK,
                 seed: int = RNG_SEED,
                 slash_per_invalid: float = 0.0,
                 penalty_alpha: float = 1e-3,
                 w_skipped: float = W_SKIPPED,
                 w_invalid: float = W_INVALID,
                 w_delayed: float = W_DELAYED,
                 txs_per_slot_mean: int = 1,
                 max_txs_per_block: int = 1):
        self.cfg = {
            "slots_per_epoch": int(slots_per_epoch),
            "eep_slot": int(slots_per_epoch * eep_fraction),
            "penalty_factor": float(penalty_factor),
            "recovery_factor": float(recovery_factor),
            "tolerated_bad_ratio": float(tolerated_bad_ratio),
            "reward_per_block": float(reward_per_block),
            "seed": int(seed),
            "slash_per_invalid": float(slash_per_invalid),
            "stake_scale_int": int(TOKEN_SCALE_INT),
            "penalty_alpha": float(penalty_alpha),
            "w_skipped": float(w_skipped),
            "w_invalid": float(w_invalid),
            "w_delayed": float(w_delayed),
            "txs_per_slot_mean": int(txs_per_slot_mean),
            "max_txs_per_block": int(max_txs_per_block),
        }
        self.epoch = 0

        self.mains: Dict[str, MainAddress] = {}
        self.overflows: Dict[str, OverflowNode] = {}
        self.wallets: Dict[str, Wallet] = {}

        self.pending_ops: List[Dict] = []
        self.history: List[Dict] = []

        self.weights_for_epoch: Dict[str, float] = {}
        self.weights_next_epoch: Dict[str, float] = {}

        self.current_schedule: List[str] = []
        self.next_schedule: List[str] = []

        self.current_seed: bytes = b""
        self.next_seed: bytes = b""

        self._slot_outcomes_current: List[str] = []
        self._initialized_weights = False

        # mempool e txid veloce
        self.mempool: List[Transaction] = []
        self._tx_seq = 0

        self.tx_senders_whitelist: Optional[set[str]] = None
        self.tx_recipients_whitelist: Optional[set[str]] = None
        self.tx_min_fee_kind: str = "PAY"

    # ---------------------------
    # Registrazione
    # ---------------------------
    def register_wallet(self, w: Wallet):
        self.wallets[w.address] = w

    def register_main(self, m: MainAddress):
        self.mains[m.address] = m

    def index_overflow(self, node: OverflowNode):
        if node.main_id not in self.mains:
            raise ValueError("main_id non registrato")
        self.overflows[node.address] = node

    # ---------------------------
    # Operazioni differite (EEP)
    # ---------------------------
    def queue_bet(self, from_addr: str, to_main_id: str, amount: float, when_slot: int):
        if to_main_id not in self.mains:
            raise ValueError(f"Main non registrato: {to_main_id}")
        if from_addr not in self.wallets and from_addr != "_self_":
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        if amount < MIN_BET_AMOUNT:
            raise ValueError(f"Importo minimo di puntata è {MIN_BET_AMOUNT}, got {amount}")
        self.pending_ops.append({
            "kind": "STAKE",
            "from": from_addr,
            "to": to_main_id,
            "amount": float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_stake_undo(self, from_addr: str, when_slot: int):
        if from_addr not in self.wallets:
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        self.pending_ops.append({
            "kind": "STAKE_UNDO",
            "from": from_addr,
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_self_unbet(self, main_id: str, when_slot: int, amount: Optional[float] = None):
        if main_id not in self.mains:
            raise ValueError(f"Main non registrato: {main_id}")
        self.pending_ops.append({
            "kind": "SELF_UNBET",
            "to": main_id,
            "amount": None if amount is None else float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def _apply_ops_at_current_eep(self):
        eep = self.cfg["eep_slot"]
        eligible, keep = [], []
        ep = self.epoch
        for op in self.pending_ops:
            if (op["epoch"] == ep and op["slot"] <= eep) or (op["epoch"] == ep - 1 and op["slot"] > eep):
                eligible.append(op)
            else:
                keep.append(op)

        def op_key(op):
            return (0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
                    op.get("epoch", -1), op.get("slot", -1), op.get("from", ""), op.get("to", ""))

        eligible.sort(key=op_key)

        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]
                st = self.wallets.get(deleg_addr)
                if not st: continue
                for m in self.mains.values():
                    cur = m.delegates.get(deleg_addr, 0.0)
                    if cur > 0:
                        st.green += cur
                        m.delegates.pop(deleg_addr, None)

            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if not to_main: continue
                amount = float(op["amount"])
                if amount < MIN_BET_AMOUNT: continue
                from_addr = op["from"]
                if from_addr == "_self_":
                    if amount <= to_main.green:
                        to_main.green -= amount
                        to_main.bet("_self_", amount)
                else:
                    st = self.wallets.get(from_addr)
                    if st and amount <= st.green:
                        st.green -= amount
                        to_main.bet(from_addr, amount)

            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main:
                    amt = op.get("amount", None)
                    to_main.self_unbet(amt)

        self.pending_ops = keep

    def _snapshot_all_mains_at_eep(self):
        for m in self.mains.values():
            m.snapshot_at_eep()

    # ---------------------------
    # Seed & VRF (deterministico)
    # ---------------------------
    @staticmethod
    def _int_addr(addr: str) -> int:
        return int.from_bytes(hashlib.sha256(addr.encode()).digest(), "big", signed=False)

    def _build_intervals_from_weights_vrf(self, weights: Dict[str, float]) -> Tuple[List[str], List[int], int]:
        scale = int(self.cfg["stake_scale_int"])
        items = []
        for addr, w in weights.items():
            wint = int((0.0 if w is None else float(w)) * scale)
            if wint <= 0: continue
            items.append((addr, self._int_addr(addr), wint))
        items.sort(key=lambda x: x[1])
        addresses, prefix_ends, acc = [], [], 0
        for addr, _, wint in items:
            addresses.append(addr); acc += wint; prefix_ends.append(acc)
        return addresses, prefix_ends, acc

    @staticmethod
    def _hash_chain(seed: bytes, n: int):
        """Generatore di n digest consecutivi: x1=H(seed), x{i+1}=H(xi)."""
        x = hashlib.sha256(seed).digest()
        for _ in range(n):
            yield x
            x = hashlib.sha256(x).digest()

    def _vrf_schedule_exact(self, seed: bytes, weights: Dict[str, float]) -> Tuple[Dict[str, int], List[str]]:
        S = self.cfg["slots_per_epoch"]
        addrs, prefix_ends, total_int = self._build_intervals_from_weights_vrf(weights)
        if total_int <= 0 or not addrs:
            return {addr: 0 for addr in self.overflows}, []
        counts = {addr: 0 for addr in self.overflows}
        schedule: List[str] = []
        for x in self._hash_chain(seed, S):
            ticket = int.from_bytes(x, "big") % total_int
            j = bisect.bisect_left(prefix_ends, ticket)
            if j >= len(addrs): j = len(addrs) - 1
            addr = addrs[j]
            schedule.append(addr)
            counts[addr] = counts.get(addr, 0) + 1
        return counts, schedule

    def _epoch_seed_str(self) -> str:
        return f"takamaka-seed:{self.cfg['seed']}:{self.epoch}"

    # ---------------------------
    # Pesi effettivi
    # ---------------------------
    def _compute_effective_weights_now(self) -> Dict[str, float]:
        weights: Dict[str, float] = {}
        alpha = self.cfg["penalty_alpha"]
        for node in self.overflows.values():
            main = self.mains[node.main_id]
            resid = node.penalty_slots_residual
            eff = main.active_stake / (1.0 + alpha * (resid if resid > 0 else 0))
            node.penalty_score = alpha * (resid if resid > 0 else 0)
            weights[node.address] = eff
        return weights

    # ---------------------------
    # Penalità
    # ---------------------------
    def _apply_penalties(self, per_node_outcomes: Dict[str, Dict[str, int]]):
        wS = self.cfg["w_skipped"]; wI = self.cfg["w_invalid"]; wD = self.cfg["w_delayed"]
        PF = self.cfg["penalty_factor"]; RF = self.cfg["recovery_factor"]; tol_ratio = self.cfg["tolerated_bad_ratio"]
        for addr, outcome in per_node_outcomes.items():
            node = self.overflows[addr]
            assigned = max(1, node.assigned_slots)
            ok      = outcome.get("ok", 0)
            skipped = outcome.get("skipped", 0)
            invalid = outcome.get("invalid", 0)
            delayed = outcome.get("delayed", 0)
            tol = int(tol_ratio * assigned)
            bad_weighted = (wS * skipped) + (wI * invalid) + (wD * delayed)
            gained = max(0.0, bad_weighted - tol) * PF
            resid_prev = node.penalty_slots_residual
            resid_next = max(0, int(resid_prev) - int(RF * ok)) + int(math.ceil(gained))
            node.penalty_slots_residual = int(resid_next)
            node.penalty_score = self.cfg["penalty_alpha"] * node.penalty_slots_residual
            spiv = self.cfg["slash_per_invalid"]
            if spiv > 0.0 and invalid > 0:
                node.slash_green(invalid * spiv)

    # ---------------------------
    # Mempool & transazioni
    # ---------------------------
    def _wallet_by_addr(self, addr: str) -> Optional[Wallet]:
        return self.wallets.get(addr) or self.mains.get(addr) or self.overflows.get(addr)

    def set_tx_participants(self, *, senders: Optional[list[str]] = None,
                        recipients: Optional[list[str]] = None) -> None:
        """Limita la generazione di transazioni a questi insiemi (opzionali)."""
        self.tx_senders_whitelist   = set(senders)    if senders    else None
        self.tx_recipients_whitelist = set(recipients) if recipients else None

    def _gen_tx_batch(self, rng, n: int) -> List[Transaction]:
        if n <= 0:
            return []

        # 1) fee minima attesa
        fee_kind = getattr(self, "tx_min_fee_kind", "PAY")
        fee_min = float(FEE_SCHEDULE_TKR.get(fee_kind, 0.0))


        # 2) costruisci le liste di mittenti/recipienti eleggibili
        def _eligible_senders() -> list[str]:
            # parte da tutti i wallet o whitelist
            base = self.tx_senders_whitelist or set(self.wallets.keys())
            out = []
            for addr in base:
                w = self.wallets.get(addr)
                if not w:
                    continue
                # deve potere pagare almeno la fee con red+green
                if (w.red + w.green) >= fee_min:
                    out.append(addr)
            return out

        def _eligible_recipients() -> list[str]:
            base = self.tx_recipients_whitelist or set(self.wallets.keys())
            # nessun requisito per i recipienti (possono ricevere anche a 0)
            return [a for a in base if a in self.wallets]

        senders = _eligible_senders()
        recipients = _eligible_recipients()

        if len(senders) == 0 or len(recipients) == 0:
            return []
        # deve esserci almeno 1 destinatario diverso dal mittente; più è meglio
        if len(senders) == 1 and len(recipients) == 1 and (list(senders)[0] == list(recipients)[0]):
            return []

        txs: list[Transaction] = []
        choice = rng.choice
        rnd = rng.random

        for _ in range(n):
            s = choice(senders)
            t = choice(recipients)
            # evita self-transfer puro; se capita, riprova 3 volte
            tries = 0
            while t == s and tries < 3:
                t = choice(recipients); tries += 1
            if s == t:
                continue

            # payload "realistico": non superare i saldi del mittente
            wsrc = self.wallets[s]
            # scegli importi che non sforano
            # prob. 30% green, 70% red come nel tuo generatore originale
            amt_g = (rnd() * min(5.0, wsrc.green)) if rnd() < 0.3 else 0.0
            amt_r = (rnd() * min(5.0, wsrc.red))   if rnd() < 0.7 else 0.0

            # se fee_min > 0 e il mittente non ha capienza, salta
            if (wsrc.red + wsrc.green) < fee_min:
                continue

            txid = f"{self.epoch}:{self._tx_seq}"; self._tx_seq += 1
            txs.append(Transaction(
                txid=txid, kind="PAY", sender=s, recipient=t,
                amount_green=float(amt_g), amount_red=float(amt_r), valid=True
            ))

        return txs


    @staticmethod
    def _calc_required_fee_tkr(tx: Transaction) -> float:
        return FEE_SCHEDULE_TKR.get(tx.kind, 0.0)

    def _pay_fee_and_apply_payload(self, tx: Transaction, *, dry_run: bool = False) -> bool:
        """Check e (se commit) detrazione fee e payload PAY.
           Fee accreditate al Main solo a blocco valido (commit a fine slot)."""
        src = self._wallet_by_addr(tx.sender); dst = self._wallet_by_addr(tx.recipient)
        if not src or not dst: return False
        required = self._calc_required_fee_tkr(tx)
        paid = float(getattr(tx, "fee_paid_tkr", 0.0))
        if paid + 1e-18 < required: return False

        # capienza fee senza mutare
        need = required
        can_r = min(src.red, need); need -= can_r
        if need > 0:
            can_g = min(src.green, need); need -= can_g
        if need > 0: return False

        # payload PAY (check)
        if tx.kind == "PAY":
            if src.green < tx.amount_green or src.red < tx.amount_red:
                return False

        if dry_run:
            return True

        # commit fee: TKR -> fallback TKG
        need = required
        take_r = min(src.red, need); src.red -= take_r; need -= take_r
        if need > 0:
            take_g = min(src.green, need); src.green -= take_g; need -= take_g
        if need > 0:
            return False

        # commit payload
        if tx.kind == "PAY":
            if tx.amount_green > 0:
                src.green -= tx.amount_green; dst.green += tx.amount_green
            if tx.amount_red > 0:
                src.red   -= tx.amount_red;   dst.red   += tx.amount_red
        return True

    # ---------------------------
    # Next seed
    # ---------------------------
    def _compute_next_seed_from_eep_block(self) -> bytes:
        eep_idx = max(0, self.cfg["eep_slot"] - 1)
        chosen_slot = None
        if self._slot_outcomes_current:
            i = min(eep_idx, len(self._slot_outcomes_current) - 1)
            while i >= 0:
                if self._slot_outcomes_current[i] == "ok":
                    chosen_slot = i; break
                i -= 1
        if chosen_slot is None:
            return hashlib.sha256(b"fallback|seed|" + str(self.epoch).encode()).digest()
        producer_addr = self.current_schedule[chosen_slot]
        block_message = (b"|epoch|" + self.epoch.to_bytes(8, "big") +
                         b"|slot|" + chosen_slot.to_bytes(8, "big") +
                         b"|producer|" + producer_addr.encode())
        return hashlib.sha256(block_message).digest()

    def _prepare_next_epoch_state(self):
        self._apply_ops_at_current_eep()
        self._snapshot_all_mains_at_eep()
        self.weights_next_epoch = self._compute_effective_weights_now()
        self.next_seed = self._compute_next_seed_from_eep_block()
        _, self.next_schedule = self._vrf_schedule_exact(self.next_seed, self.weights_next_epoch)

    def simulate_one_epoch(self) -> Dict:
        cfg = self.cfg
        S = cfg["slots_per_epoch"]
        max_txs_per_block = cfg["max_txs_per_block"]

        # ============================
        # Bootstrap epoca 0 (con schedule e slot, ma NO TX auto)
        # ============================
        if self.epoch == 0 and not self._initialized_weights:
            # Applica operazioni fino all'EEP e fotografa le deleghe
            self._apply_ops_at_current_eep()
            self._snapshot_all_mains_at_eep()

            # Inizializza pesi e schedule per epoca 0
            self.weights_for_epoch = self._compute_effective_weights_now()
            self._initialized_weights = True
            self.current_seed = hashlib.sha256(b"genesis-seed|" + str(cfg['seed']).encode()).digest()
            _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)

            # Aggiorna stake effettiva
            for addr, node in self.overflows.items():
                node.effective_stake = float(self.weights_for_epoch.get(addr, 0.0))

            # *** NIENTE generazione mempool in epoca 0 ***
            # Usiamo la mempool così com'è (tipicamente vuota)
            mp = self.mempool  # non la resetto: se volessi pre-caricare tx manualmente, verranno lette
            take_n = max_txs_per_block
            calc_fee = self._calc_required_fee_tkr
            pay_apply = self._pay_fee_and_apply_payload

            per_node_outcomes = _dd(lambda: {"ok": 0, "skipped": 0, "invalid": 0, "delayed": 0})
            fees_epoch_by_main = _dd(float)
            self._slot_outcomes_current = []
            rng_slots = _rnd.Random(cfg["seed"] * 7919 + self.epoch)
            slots_per_node = {addr: 0 for addr in self.overflows}

            # Processiamo comunque gli slot (blocco vuoto => sempre validabile se outcome=="ok")
            for slot_idx, addr in enumerate(self.current_schedule):
                node = self.overflows[addr]
                main = self.mains[node.main_id]
                slots_per_node[addr] = slots_per_node.get(addr, 0) + 1

                outcome = node.behavior_outcome(rng_slots)
                block_valid = (outcome == "ok")
                collected_fees_tkr = 0.0

                if outcome != "skipped":
                    picked = mp[:take_n]; del mp[:take_n]
                    prepared = []
                    if block_valid:
                        # DRY-RUN forte (anche se tipicamente non ci sono tx)
                        temp_bal = {}
                        def _get_temp(a):
                            if a not in temp_bal:
                                w = self._wallet_by_addr(a)
                                if not w: return None
                                temp_bal[a] = {"g": float(getattr(w, "green", 0.0)),
                                            "r": float(getattr(w, "red", 0.0))}
                            return temp_bal[a]

                        for tx in picked:
                            required = calc_fee(tx)
                            tx.valid = True
                            tx.fee_paid_tkr = required
                            if float(tx.fee_paid_tkr) + 1e-18 < required:
                                block_valid = False; break
                            src = self._wallet_by_addr(tx.sender)
                            dst = self._wallet_by_addr(tx.recipient)
                            if not src or not dst:
                                block_valid = False; break
                            tb = _get_temp(tx.sender)
                            if tb is None:
                                block_valid = False; break
                            # Fee
                            need = required
                            dr = min(tb["r"], need); tb["r"] -= dr; need -= dr
                            if need > 0:
                                dg = min(tb["g"], need); tb["g"] -= dg; need -= dg
                            if need > 0:
                                block_valid = False; break
                            # Payload
                            ag = float(getattr(tx, "amount_green", 0.0))
                            ar = float(getattr(tx, "amount_red", 0.0))
                            if tb["g"] < ag or tb["r"] < ar:
                                block_valid = False; break
                            tb["g"] -= ag; tb["r"] -= ar
                            prepared.append((tx, required))

                        if block_valid:
                            for tx, required in prepared:
                                assert pay_apply(tx, dry_run=False)
                                collected_fees_tkr += required

                if block_valid and outcome == "ok":
                    per_node_outcomes[addr]["ok"] += 1
                    self._slot_outcomes_current.append("ok")
                    main.credit_reward(cfg["reward_per_block"])
                    if collected_fees_tkr > 0.0:
                        main.deposit(red=collected_fees_tkr)
                        fees_epoch_by_main[main.address] += collected_fees_tkr
                else:
                    if outcome in ("invalid", "delayed"):
                        per_node_outcomes[addr][outcome] += 1
                    elif outcome == "skipped":
                        per_node_outcomes[addr]["skipped"] += 1
                    else:
                        per_node_outcomes[addr]["invalid"] += 1
                    self._slot_outcomes_current.append("invalid" if outcome == "ok" else outcome)

            for addr, node in self.overflows.items():
                node.assigned_slots = slots_per_node.get(addr, 0)

            # Prepara epoca 1
            self._apply_penalties(per_node_outcomes)
            self._prepare_next_epoch_state()

            total_effective_stake = sum(self.weights_for_epoch.values())
            kpi = {
                "epoch": self.epoch,
                "slots_per_node": dict(slots_per_node),
                "per_node_outcomes": {addr: dict(v) for addr, v in per_node_outcomes.items()},
                "penalty_slots_residual": {addr: self.overflows[addr].penalty_slots_residual for addr in self.overflows},
                "penalties_alpha_scaled": {addr: self.overflows[addr].penalty_score for addr in self.overflows},
                "effective_stake_per_overflow": {addr: self.overflows[addr].effective_stake for addr in self.overflows},
                "total_effective_stake": total_effective_stake,
                "total_slots": int(cfg["slots_per_epoch"]),
                "fees_epoch_by_main": dict(fees_epoch_by_main)
            }
            self.history.append(kpi)

            # Rollover → epoca 1
            self.weights_for_epoch = dict(self.weights_next_epoch)
            self.current_seed = self.next_seed
            self.current_schedule = list(self.next_schedule)
            self.epoch += 1
            return kpi

        # ============================
        # Epoche >= 1 (NO TX auto)
        # ============================
        if not self.current_schedule:
            if not self._initialized_weights:
                self.weights_for_epoch = self._compute_effective_weights_now()
                self._initialized_weights = True
            self.current_seed = hashlib.sha256(self._epoch_seed_str().encode() + b"|current-seed|").digest()
            _, self.current_schedule = self._vrf_schedule_exact(self.current_seed, self.weights_for_epoch)

        for addr, node in self.overflows.items():
            node.effective_stake = float(self.weights_for_epoch.get(addr, 0.0))

        per_node_outcomes = _dd(lambda: {"ok": 0, "skipped": 0, "invalid": 0, "delayed": 0})
        fees_epoch_by_main = _dd(float)
        self._slot_outcomes_current = []

        rng_slots = _rnd.Random(cfg["seed"] * 7919 + self.epoch)

        # *** NIENTE generazione casuale ***
        mp = self.mempool  # usa ciò che eventualmente hai caricato tu
        take_n = max_txs_per_block
        calc_fee = self._calc_required_fee_tkr
        pay_apply = self._pay_fee_and_apply_payload
        slots_per_node = {addr: 0 for addr in self.overflows}

        for slot_idx, addr in enumerate(self.current_schedule):
            node = self.overflows[addr]
            main = self.mains[node.main_id]
            slots_per_node[addr] = slots_per_node.get(addr, 0) + 1

            outcome = node.behavior_outcome(rng_slots)
            block_valid = (outcome == "ok")
            collected_fees_tkr = 0.0

            if outcome != "skipped":
                picked = mp[:take_n]
                del mp[:take_n]
                prepared: List[Tuple[Transaction, float]] = []
                if block_valid:
                    # DRY-RUN forte
                    temp_bal = {}
                    def _get_temp(a):
                        if a not in temp_bal:
                            w = self._wallet_by_addr(a)
                            if not w: return None
                            temp_bal[a] = {"g": float(getattr(w, "green", 0.0)),
                                        "r": float(getattr(w, "red", 0.0))}
                        return temp_bal[a]

                    used_senders = set()  # opzionale: max 1 tx per sender per blocco

                    for tx in picked:
                        required = calc_fee(tx)
                        # Non manipoliamo tx.valid/fee qui: le rispettiamo come sono state impostate
                        if not getattr(tx, "valid", True):
                            block_valid = False; break
                        fee_paid = float(getattr(tx, "fee_paid_tkr", required))
                        if fee_paid + 1e-18 < required:
                            block_valid = False; break

                        src = self._wallet_by_addr(tx.sender)
                        dst = self._wallet_by_addr(tx.recipient)
                        if not src or not dst:
                            block_valid = False; break

                        if tx.sender in used_senders:
                            continue
                        used_senders.add(tx.sender)

                        tb = _get_temp(tx.sender)
                        if tb is None:
                            block_valid = False; break

                        # Fee
                        need = required
                        dr = min(tb["r"], need); tb["r"] -= dr; need -= dr
                        if need > 0:
                            dg = min(tb["g"], need); tb["g"] -= dg; need -= dg
                        if need > 0:
                            block_valid = False; break

                        # Payload
                        ag = float(getattr(tx, "amount_green", 0.0))
                        ar = float(getattr(tx, "amount_red", 0.0))
                        if tb["g"] < ag or tb["r"] < ar:
                            block_valid = False; break

                        tb["g"] -= ag; tb["r"] -= ar
                        prepared.append((tx, required))

                    if block_valid:
                        for tx, required in prepared:
                            assert pay_apply(tx, dry_run=False)
                            collected_fees_tkr += required

            if block_valid and outcome == "ok":
                per_node_outcomes[addr]["ok"] += 1
                self._slot_outcomes_current.append("ok")
                main.credit_reward(cfg["reward_per_block"])
                if collected_fees_tkr > 0.0:
                    main.deposit(red=collected_fees_tkr)
                    fees_epoch_by_main[main.address] += collected_fees_tkr
            else:
                if outcome in ("invalid", "delayed"):
                    per_node_outcomes[addr][outcome] += 1
                elif outcome == "skipped":
                    per_node_outcomes[addr]["skipped"] += 1
                else:
                    per_node_outcomes[addr]["invalid"] += 1
                self._slot_outcomes_current.append("invalid" if outcome == "ok" else outcome)

        for addr, node in self.overflows.items():
            node.assigned_slots = slots_per_node.get(addr, 0)

        self._apply_penalties(per_node_outcomes)
        self._prepare_next_epoch_state()

        total_effective_stake = sum(self.weights_for_epoch.values())
        kpi = {
            "epoch": self.epoch,
            "slots_per_node": dict(slots_per_node),
            "per_node_outcomes": {addr: dict(v) for addr, v in per_node_outcomes.items()},
            "penalty_slots_residual": {addr: self.overflows[addr].penalty_slots_residual for addr in self.overflows},
            "penalties_alpha_scaled": {addr: self.overflows[addr].penalty_score for addr in self.overflows},
            "effective_stake_per_overflow": {addr: self.overflows[addr].effective_stake for addr in self.overflows},
            "total_effective_stake": total_effective_stake,
            "total_slots": int(cfg["slots_per_epoch"]),
            "fees_epoch_by_main": dict(fees_epoch_by_main)
        }
        self.history.append(kpi)

        # Rollover
        self.weights_for_epoch = dict(self.weights_next_epoch)
        self.current_seed = self.next_seed
        self.current_schedule = list(self.next_schedule)
        self.epoch += 1
        return kpi




    # ---------------------------
    # Report per epoca
    # ---------------------------
    def summary_all_epochs_df(self) -> pd.DataFrame:
        ok_cum_map = _dd(int)
        fees_cum_by_main = _dd(float)

        rows = []
        S = self.cfg["slots_per_epoch"]
        reward_per_block = float(self.cfg.get("reward_per_block", 0.0))

        for kpi in self.history:
            total_effective = kpi.get("total_effective_stake", 0.0)
            total_slots_epoch = int(kpi.get("total_slots", 0))

            for addr, o in kpi.get("per_node_outcomes", {}).items():
                ok_cum_map[addr] += int(o.get("ok", 0))
            for m, f in kpi.get("fees_epoch_by_main", {}).items():
                fees_cum_by_main[m] += float(f)

            for addr, slots in kpi["slots_per_node"].items():
                o = kpi["per_node_outcomes"].get(addr, {})
                node = self.overflows[addr]
                main = self.mains[node.main_id]

                ok = int(o.get("ok", 0))
                skipped = int(o.get("skipped", 0))
                invalid = int(o.get("invalid", 0))
                delayed = int(o.get("delayed", 0))
                bad_abs = skipped + invalid + delayed

                stake_share_pct = (node.effective_stake / total_effective * 100.0) if total_effective > 0 else 0.0
                delivered_slots_share_pct = (ok / total_slots_epoch * 100.0) if total_slots_epoch > 0 else 0.0
                bad_slots_pct_node = (bad_abs / slots * 100.0) if slots > 0 else 0.0

                coinbase_epoch = ok * reward_per_block
                fees_epoch = float(kpi.get("fees_epoch_by_main", {}).get(main.address, 0.0))
                reward_epoch_total_value = coinbase_epoch

                coinbase_cum = ok_cum_map[addr] * reward_per_block
                fees_cum_main = fees_cum_by_main[main.address]

                rows.append({
                    "epoch": kpi["epoch"],
                    "overflow": addr,
                    "main": main.address,
                    "assigned_slots": int(slots),
                    "share_assigned_slots": round((int(slots) / S) * 100, 3),

                    "ok": ok, "skipped": skipped, "invalid": invalid, "delayed": delayed,
                    "bad_slots_abs": int(bad_abs), "bad_slots_pct_node": round(bad_slots_pct_node, 2),

                    "effective_stake": node.effective_stake,
                    "stake_share_pct": round(stake_share_pct, 2),
                    "delivered_slots_share_pct": round(delivered_slots_share_pct, 2),

                    "coinbase_epoch_TKG": round(coinbase_epoch, 6),
                    "fees_epoch_TKR_at_main": round(fees_epoch, 6),
                    "reward_epoch_total_value_TKG_only": round(reward_epoch_total_value, 6),

                    "coinbase_cum_TKG": round(coinbase_cum, 6),
                    "fees_cum_TKR_at_main": round(fees_cum_main, 6),

                    "penalty_slots_residual": getattr(node, "penalty_slots_residual", 0),
                    "penalty_score": node.penalty_score,
                })

        return pd.DataFrame(rows)


# Simulations

## Simulation 1

This simulation runs for 50 epochs with a large mempool of randomly generated transactions per slot.
- Initial stake distribution: Node 1 = 40%, Node 2 = 30%, Node 3 = 30% (delegations at epoch 0).
- Behavior: all three nodes are honest; every assigned slot is produced as ok (no skips/invalid/delayed).
- Expected outcome: the scheduler allocates slots ≈ proportionally to stake (≈40/30/30); coinbase (TKG) and fees (TKR) follow the same proportions up to small stochastic noise; no penalties accumulate.

In [9]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,
    seed=RNG_SEED
)

# main (wallet)
m1 = MainAddress(address="MAIN_1", green=0.0, red=0.0)
m2 = MainAddress(address="MAIN_2", green=0.0, red=0.0)
m3 = MainAddress(address="MAIN_3", green=0.0, red=0.0)

# registra come main
sim.register_main(m1)
sim.register_main(m2)
sim.register_main(m3)

# overflow onesti
o1 = OverflowNode(address="OV_1", green=0.0, red=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o2 = OverflowNode(address="OV_2", green=0.0, red=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o3 = OverflowNode(address="OV_3", green=0.0, red=0.0, p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)

# registra overflow e collega ai main e indicizza
m1.register_overflow(o1); sim.index_overflow(o1)
m2.register_overflow(o2); sim.index_overflow(o2)
m3.register_overflow(o3); sim.index_overflow(o3)

# stakeholders con stake 40/30/30
s1 = Wallet(address="STK_1", green=400.0, red=50.0)
s2 = Wallet(address="STK_2", green=300.0, red=50.0)
s3 = Wallet(address="STK_3", green=300.0, red=50.0)

# registra stakeholder come wallet
sim.register_wallet(s1)
sim.register_wallet(s2)
sim.register_wallet(s3)

# Wallets to simulate transactions
w1 = Wallet(address="W_1", green=400.0, red=50.0)
w2 = Wallet(address="W_2", green=300.0, red=50.0)
w3 = Wallet(address="W_3", green=300.0, red=50.0)

sim.register_wallet(w1)
sim.register_wallet(w2)
sim.register_wallet(w3)

sim.set_tx_participants(senders=["W_1","W_2","W_3"], recipients=["STK_1","STK_2","STK_3"])

# deleghe epoch 0 (slot 0)
sim.queue_bet(from_addr="STK_1", to_main_id="MAIN_1", amount=400.0, when_slot=0)
sim.queue_bet(from_addr="STK_2", to_main_id="MAIN_2", amount=300.0, when_slot=0)
sim.queue_bet(from_addr="STK_3", to_main_id="MAIN_3", amount=300.0, when_slot=0)

for _ in range(50):
    sim.simulate_one_epoch()

# Un solo DataFrame alla fine
summary_baseline = sim.summary_all_epochs_df()
summary_baseline.to_csv("./data/scenario1_baseline.csv", index=False)


## Simulation 2

Same setup as Simulation 1 (stake 40/30/30, rich mempool, 50 epochs), but with transient malicious behavior for Node 1:
- Epochs 0–19: Node 1 behaves maliciously (e.g., corrupt/underpaid transactions via p_bad_tx=1.0), so its blocks are marked invalid, fees are not collected, and penalties increase its residual, reducing its effective stake and slot share.
- Epochs 20–49: Node 1 returns to honest behavior; recovery reduces residual penalties over time and its effective stake/slot share gradually restores.
- Expected outcome: during 0–19 Node 1’s valid blocks and rewards drop sharply; from 20–49 they recover progressively, illustrating long-term penalization and recovery dynamics.

In [ ]:
sim = TakamakaSim(
    slots_per_epoch=SLOTS_PER_EPOCH,
    seed=RNG_SEED
)

# main (wallet)
m1 = MainAddress(address="MAIN_1", green=0.0, red=0.0)
m2 = MainAddress(address="MAIN_2", green=0.0, red=0.0)
m3 = MainAddress(address="MAIN_3", green=0.0, red=0.0)

# registra come main
sim.register_main(m1)
sim.register_main(m2)
sim.register_main(m3)

# overflow (inizialmente tutti onesti)
o1 = OverflowNode(address="OV_1", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o2 = OverflowNode(address="OV_2", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)
o3 = OverflowNode(address="OV_3", green=0.0, red=0.0,
                  p_skip=0.0, p_invalid=0.0, p_delayed=0.0, p_bad_tx=0.0)

# registra overflow e collega ai main
m1.register_overflow(o1); sim.index_overflow(o1)
m2.register_overflow(o2); sim.index_overflow(o2)
m3.register_overflow(o3); sim.index_overflow(o3)

# stakeholders con stake 40/30/30
s1 = Wallet(address="STK_1", green=400.0, red=50.0)
s2 = Wallet(address="STK_2", green=300.0, red=50.0)
s3 = Wallet(address="STK_3", green=300.0, red=50.0)

# registra stakeholder come wallet
sim.register_wallet(s1)
sim.register_wallet(s2)
sim.register_wallet(s3)

# wallets per generare transazioni
w1 = Wallet(address="W_1", green=400.0, red=50.0)
w2 = Wallet(address="W_2", green=300.0, red=50.0)
w3 = Wallet(address="W_3", green=300.0, red=50.0)

sim.register_wallet(w1)
sim.register_wallet(w2)
sim.register_wallet(w3)

sim.set_tx_participants(
    senders=["W_1", "W_2", "W_3"],
    recipients=["STK_1", "STK_2", "STK_3"]
)

# deleghe epoch 0 (slot 0)
sim.queue_bet(from_addr="STK_1", to_main_id="MAIN_1", amount=400.0, when_slot=0)
sim.queue_bet(from_addr="STK_2", to_main_id="MAIN_2", amount=300.0, when_slot=0)
sim.queue_bet(from_addr="STK_3", to_main_id="MAIN_3", amount=300.0, when_slot=0)

# Simulazione 50 epoche
for ep in range(50):
    # Node 1 malevolo nelle prime 20 epoche
    if ep < 20:
        o1.p_bad_tx = 1.0    # tutti i blocchi corrotti/sottopagati
    else:
        o1.p_bad_tx = 0.0    # torna onesto

    sim.simulate_one_epoch()

# Salvataggio risultati
summary_malicious = sim.summary_all_epochs_df()
summary_malicious.to_csv("./data/scenario2_malicious.csv", index=False)
